In [ ]:
import sqlite3
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.SimpleEnum.Enumerator import EnumerateReaction
from rdkit.Chem.rdChemReactions import ReactionFromSmarts, Compute2DCoordsForReaction


In [ ]:
DATA_DIR = Path('..', 'data').resolve()
DB_PATH = DATA_DIR / 'db' / '50k_project.db'

In [ ]:
con = sqlite3.connect(DB_PATH)
cur = con.cursor()

In [ ]:
# retrieve one of each building block kind for testing
buildingblocks = cur.execute('SELECT SMILES FROM building_blocks WHERE long IN (?, ?, ?, ?, ?, ?, ?, ?);', ('Al001', 'Fused002', 'Spiro001', 'Spiro002', 'Mon001', 'Mon072', 'TerABT001', 'TerTH001')).fetchall()
names = ['initiator', 'monomer_fused', 'monomer_spiro_3', 'monomer_spiro_2', 'monomer_sub_3', 'monomer_sub_2', 'ter_abt', 'ter_th']
buildingblocks = [Chem.MolFromSmiles(sm) for tup in buildingblocks for sm in tup]
buildingblocks = {k: v for k, v in zip(names, buildingblocks)}
buildingblocks


In [ ]:
# define the reactions

rxn_TH = ReactionFromSmarts(
    'F[B-](F)(F)[#6](-[*:1])=O.O=[#6]1-[#8]C2([#6]-[#6]-[#6]-[#6]-[#6]2)[#8]C11[#6:3]-[#6:2]-[#7]-[#8]1.[#6:4]-[#6](=S)-[#7]-[#7]>>[#6:4]-[#6]-1=[#7]-[#7]=[#6](-[#6:3]-[#6:2]-[#7]-[#6](-[*:1])=O)-[#16]-1.[#6:4]-[#6]-1=[#7]-[#7]C([#6:3]-[#6:2]-[#7]-[#6](-[*:1])=O)([#16]-1)[#6](-[#8])=O.[#6:4]-[#6]-1=[#7]-[#7+]2=[#6](-[*:1])-[#7]-[#6:2]-[#6:3]C2([#16]-1)[#6](-[#8-])=O.[#6:4]-[#6]-1=[#7]-[#7]=[#6](-[*:1])-[#16]-1.[#6:4]-[#6]-1=[#7]-[#7]=[#6](-[#6:4])-[#16]-1.[#8]-[#6](=O)-[#6](=O)-[#6:3]-[#6:2]-[#7]-[#6](-[*:1])=O.[#8]-[#6](=O)-[#6:3]-[#6:2]-[#7]-[#6](-[*:1])=O.[#6:4]-[#6]-1=[#7]-[#7]=[#6](-[#16]-1)-[#6:3]=[#6:2]'
)

rxn_ABT = ReactionFromSmarts(
    "F[B-](F)(F)[#6](-[*:1])=O.O=[#6]1-[#8]C2([#6]-[#6]-[#6]-[#6]-[#6]2)[#8]C11[#6:3]-[#6:2]-[#7]-[#8]1.[#7]-c1[c:4][c:5][c:6][c:7]c1-[#16]>>[*:1]-[#6](=O)-[#7]-[#6:2]-[#6:3]-c1nc2[c:4][c:5][c:6][c:7]c2s1.[#8]-[#6](=O)C1([#6:3]-[#6:2]-[#7]-[#6](-[*:1])=O)[#7]-c2[c:4][c:5][c:6][c:7]c2-[#16]1.[#8-]-[#6](=O)C12[#6:3]-[#6:2]-[#7]-[#6](-[*:1])=[#7+]1-c1[c:4][c:5][c:6][c:7]c1-[#16]2.[*:1]-c1nc2[c:4][c:5][c:6][c:7]c2s1.[#7]-c1[c:4][c:5][c:6][c:7]c1-[#16]-[#16]-c1[c:7][c:6][c:5][c:4]c1-[#7].[#8]-[#6](=O)-[#6](=O)-[#6:3]-[#6:2]-[#7]-[#6](-[*:1])=O.[#8]-[#6](=O)-[#6:3]-[#6:2]-[#7]-[#6](-[*:1])=O.[#6:2]=[#6;v4:3]-c1nc2[c:4][c:5][c:6][c:7]c2s1"
)
# prepare for visualization
Compute2DCoordsForReaction(rxn_TH)
Compute2DCoordsForReaction(rxn_ABT)
# prepare for enumeration
rxn_TH.Initialize()
rxn_ABT.Initialize()

In [ ]:
# Control reactions visually
Draw.ReactionToImage(rxn_TH)

In [ ]:
# Control reactions visually
Draw.ReactionToImage(rxn_ABT)

In [ ]:
# Control which reactants work
print('Reactants working in the TH reaction:')
for k, v in buildingblocks.items():
    print(f'{k}: {rxn_TH.IsMoleculeReactant(v)}')

In [ ]:
# Control which reactants work
print('Reactants working in the ABT reaction:')
for k, v in buildingblocks.items():
    print(f'{k}: {rxn_ABT.IsMoleculeReactant(v)}')

In [ ]:
# Perform enumeration
product_generator_TH = EnumerateReaction(rxn_TH, ([buildingblocks['initiator']], [buildingblocks['monomer_fused'], buildingblocks['monomer_spiro_3'], buildingblocks['monomer_spiro_2'], buildingblocks['monomer_sub_3'], buildingblocks['monomer_sub_2']], [buildingblocks['ter_th']]), uniqueProductsOnly=True)
product_generator_ABT = EnumerateReaction(rxn_ABT, ([buildingblocks['initiator']], [buildingblocks['monomer_fused'], buildingblocks['monomer_spiro_3'], buildingblocks['monomer_spiro_2'], buildingblocks['monomer_sub_3'], buildingblocks['monomer_sub_2']], [buildingblocks['ter_abt']]), uniqueProductsOnly=True)

In [ ]:
product_list_TH = list(product_generator_TH)
product_list_ABT = list(product_generator_ABT)

In [ ]:
for ps in product_list_ABT:
    mol = ps[7]
    problems = Chem.DetectChemistryProblems(mol)
    print(problems)
    if len(problems) > 0:
        if problems[0].GetType() == 'AtomValenceException':
            try:
                atom_idx = problems[0].GetAtomIdx()
                mol.GetAtomWithIdx(atom_idx).SetNumExplicitHs(0)
            except ValueError:
                print('fail')
                pass

for reac in product_list_ABT:
    for prod in reac:
        try:
            Chem.SanitizeMol(prod)
        except:
            print('no')

In [ ]:
product_list_ABT[0][7]

In [ ]:
for atom in product_list_TH[0][0].GetAtoms():
   print(atom.GetSymbol(), atom.GetTotalValence(), atom.GetExplicitValence(), atom.GetImplicitValence(), atom.GetNumExplicitHs(), atom.GetNumImplicitHs(), atom.GetTotalNumHs(), atom.NeedsUpdatePropertyCache())

In [ ]:
# this works but better get the reaction right
product_list_TH[0][7].GetAtomWithIdx(6).SetNumExplicitHs(0)
product_list_TH[0][7]